## Notes
Switching to the new interactive plotting. It's actually much quicker like this.

In [ ]:
import logging
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)
import scanpy as sc
import scvelo as scv
import scvi
import seaborn as sns
import plotly.express as px
import numpy as np
from dash import Dash, dcc, html, Input, Output,dash_table, ctx, State
import dash_ag_grid as dag

import pandas as pd

import os
import sys
import time
import gc
os.environ['R_HOME'] = sys.exec_prefix+"/lib/R/"

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from matplotlib.lines import Line2D 

from copy import copy
reds = copy(mpl.cm.Reds)
reds.set_under("lightgray")

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []

    for k in range(pl_entries):
        C = list(map(np.uint8, np.array(cmap(k * h)[:3]) * 255))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])

    return pl_colorscale

plotly_reds = matplotlib_to_plotly(reds, 255)
plotly_reds[0] = [0.0, 'rgb(211, 211, 211)']

project_directory = '/Cranio_Lab/Louk_Seton/4_species_project'
os.chdir(os.path.expanduser("~")+project_directory)

In [ ]:
# ##read in all samples
# sample_names = ['ME8','ME9','ME10','ME11','ME12'] #specify the sample names
# species = 'mouse' #specify the species
# genome = 'mm10' #specify the genome
# output_prefix = 'h5ad_files/' #specify the location of the cellranger output

# adata_dict = {}
# for sample in sample_names:
#     adata_dict[sample] = sc.read(output_prefix+species+'/'+genome+'/'+sample+'_after_filtering.h5ad')

# leiden_res = [0.5,0.6,0.7,0.8,0.9,1,1.2,1.4,1.6,1.8,2,2.2,2.4,2.6,2.8,3]
# for sample in adata_dict:
#     for i in leiden_res:
#         sc.tl.leiden(adata_dict[sample], resolution = i, key_added = 'leiden'+str(i))
#     try:
#         del adata_dict[sample].uns['log1p']
#     except: print('no log1p')
#     for i in leiden_res:
#         sc.tl.rank_genes_groups(adata_dict[sample], 'leiden'+str(i), method='wilcoxon', 
#                                 key_added = "wilcoxon_"+'leiden'+str(i))

#     sc.tl.rank_genes_groups(adata_dict[sample], 'leiden_post_QC', method='wilcoxon', 
#                             key_added = "wilcoxon_"+'leiden_post_QC')
#     adata_dict[sample].write(output_prefix+species+'/'+genome+'/'+sample+'_interactive.h5ad')

In [ ]:
##read in all samples
sample_names = ['ME8','ME9','ME10','ME11','ME12'] #specify the sample names
species = 'mouse' #specify the species
genome = 'mm10' #specify the genome
output_prefix = 'h5ad_files/' #specify the location of the cellranger output

adata_dict = {}
for sample in sample_names:
    adata_dict[sample] = sc.read(output_prefix+species+'/'+genome+'/'+sample+'_interactive.h5ad')

In [ ]:
port = 11198
initial_gene = 'Car4'
initial_obs = 'sample'

external_stylesheets = ['https://bootswatch.com/5/flatly/bootstrap.css']

app = Dash(__name__,
           external_stylesheets=external_stylesheets,
           suppress_callback_exceptions=True)


##########################################################
######################  Display of tab 1 #################
##########################################################
tab1 = html.Div([
    html.Div([
        html.Div([
            
            
            ##############################################
            
            ##############################################
            ######  select gene to plot from list  #######
            ##############################################
            html.Label("Select gene"),
            dcc.Dropdown(
                         id="dropdown_var",
                        value = initial_gene,
                        ),
            ##############################################
            
            ##############################################
            ####other gene values to show if available####
            ##############################################
            html.Label(id='gene_id'),
            html.Br(),
            html.Label(id='gene_product'),
            html.Br(),
            html.Div([],id='db_xref'),
            html.Br(),
            ##############################################
            
            ##############################################
            #######set dot size of both umap plots########
            ##############################################
            html.Label("Point size"),
            dcc.Slider(1, 10,
                       marks = None,
                       value=3,
                       id='slider_marker_size',
                       tooltip={"placement": "bottom", "always_visible": True},
                      ),
            ##############################################
        ],
            style={'width': '49%', 'display': 'inline-block'}), #html style of left dropdown section

        
        html.Div([
            ##############################################
            ######  select obs to plot from list  #######
            ##############################################
            html.Label("Select obs"),
            dcc.Dropdown(
                         id="dropdown_obs",
                         value = initial_obs
                        ),
            ##############################################
            
        ],
           style={'width': '49%', 'float': 'right', 'display': 'inline-block'}), #html style of right dropdown section
        
    ],
        style={'padding': '10px 5px'}), #html style of dropdown section

    ##############################################
    ##########  plot left umap graph  ############
    ##############################################
    html.Div([
      dcc.Graph(id="graph1"),
        ],
        style={'width': '49%', 'display': 'inline-block'}),#html style of left graph section
    ##############################################
    
    ##############################################
    ##########  plot right umap graph  ###########
    ##############################################
    html.Div([
      dcc.Graph(id="graph2")
    ],
            style={'width': '49%', 'float': 'right', 'display': 'inline-block'}),#html style of right graph section
    ##############################################

    ##############################################
    ############### DEG table stuff ##############
    ##############################################
    html.Div([dcc.Dropdown(id = 'cluster_select'),
             ],
             style={'width': '49%', 'display': 'inline-block','display': 'none'},
             id="html-cluster_select"),
    
    html.Div([dash_table.DataTable(filter_action="native",
                                   sort_action="native",
                                   sort_mode="multi",
                                   column_selectable="single",
                                   page_action="native",
                                   page_current= 0,
                                   page_size= 10,
                                   id = 'deg_table',
                                   ),
             ],
             style={'display': 'none'},
             id='html-deg_table'),
    ##############################################
    html.Div(dcc.Store(id='gene_plotted')),
])


##########################################################
######################  Display of DP tab ################
##########################################################

####################################
#####components of left DP col######
####################################

#components that span the entire left col
html_drop_dp_obs = html.Div([html.B("Select obs:"),
                             dcc.Dropdown(id="drop_dp_obs",
                                          style={'width':'80%'},
                                         ),
                            ],
                            style = {'display': 'none'},
                            id = 'html-drop_dp_obs')

html_drop_dp_method = html.Div([html.B("Select method:"),
                                dcc.Dropdown(id="drop_dp_method",
                                             options = ['t-test', 'wilcoxon', 't-test_overestim_var'], 
                                             value = 'wilcoxon',
                                             style={'width':'80%'},
                                            ),
                               ],
                               style = {'display': 'none'},
                               id = 'html-drop_dp_method')

html_dp_minfoldchange = html.Div([html.B("min fold change: "),
                                  dcc.Input(
                                      id = 'dp_minfoldchange',
                                      type='number',
                                      value=2.5,
                                      style={'width':'25%'},
                                  ),
                                 ],
                                 style = {'display': 'none'},
                                 id = 'html-dp_minfoldchange')

html_vmin_vmax = html.Div([html.B("vmin: "),
                           dcc.Input(id = 'dp_vmin',
                                     type='number',
                                     max = 0,
                                     value=-4,
                                     style={'width':'25%'},
                                    ),
                           html.Br(),
                           html.Br(),
                           
                           html.B("vmax: "),
                           dcc.Input(id = 'dp_vmax',
                                     type='number',
                                     min = 0,
                                     value=4,
                                     style={'width':'25%'},
                                    ),
                          ],
                          style = {'display': 'none'},
                          id = 'html-dp_vmin_vmax')

#components that span the left half of the left col
html_dp_var_type = html.Div([html.B("Genes: "),
                             dcc.RadioItems(id = 'dp_var_type',
                                            options = ['Automatic DEGs', 'Specify Genes'],
                                            value = 'Automatic DEGs',
                                            style = {'font-size': 12,},
                                           ),
                            ],
                            style={'width': '49%', 'float': 'left', 'display': 'inline-block','display': 'none'},
                            id = 'html-dp_var_type')

html_dp_n_genes = html.Div([html.Br(),
                            html.B("n_genes: "),
                            dcc.Input(id = 'dp_n_genes',
                                      type='number',
                                      min = 0,
                                      step = 1,
                                      value=4,
                                      style={'width':'50%'},
                                     ),
                           ],
                           style={'width': '49%', 'float': 'left', 'display': 'inline-block','display': 'none'},
                           id = 'html-dp_n_genes')

html_dp_select_obs = html.Div([html.B("Select categories"),
                               dcc.Dropdown(id="dp_select_obs",
                            #                options = list(adata_dict[anndata].obs[drop_dp_obs].cat.categories),
                            #                value = list(adata_dict[anndata].obs[drop_dp_obs].cat.categories),
                                            multi = True,
                                           ),
                              ],
                              style={'width': '49%', 'float': 'left', 'display': 'inline-block','display': 'none'},
                              id = 'html-dp_select_obs')

html_dp_select_var = html.Div([html.B("Select genes"),
                               dcc.Dropdown(
                                   id="dp_select_var",
                        #           options = adata_dict[anndata].var.index.tolist(),
                                   multi = True,
                               ),
                              ],
                              style={'width': '49%', 'float': 'left', 'display': 'inline-block','display': 'none'},
                              id = 'html-dp_select_var')

#components that span the right half of the left col
html_dp_obs_type = html.Div([html.B("Categories: "),
                             dcc.RadioItems(id = 'dp_obs_type',
                                            options = ['Automatic order', 'Customize order'],
                                            value = 'Automatic order',
                                            style = {'font-size': 12,},
                                           ),
                            ],
                            style={'width': '49%', 'float': 'right', 'display': 'inline-block','display': 'none'},
                            id = 'html-dp_obs_type')

html_dp_obs_order = html.Div([html.Div([html.Br(),
                                        dag.AgGrid(id="dp_obs_order",
                                                   dashGridOptions={"animateRows": True,
                                                                    "enableCellTextSelection": False,
                                                                    "skipHeaderOnAutoSize": True,
                                                                    #"ensureDomOrder": True,
                                                                    "rowDragManaged": True,
                                                                    #"rowDragMultiRow": True,
                                                                    #"rowSelection": "multiple",
                                                                    #"rowDragEntireRow": True,
                                                                    "pagination": False,
                                                                    "suppressRowClickSelection": True,
                                                                   }
                                                  )
                                       ],
                                       style={'width': '49%', 'float': 'right', 'display': 'inline-block','display': 'none'},
                                       id = 'html-dp_obs_order'),
                              html.Div([dcc.Store(id='dp_obs_order_store',
                                                  data = None,
                                                 ),
                                       ])
                             ],)

####################################
####################################


####################################
#####components of right DP col#####
####################################

html_dp_figure = html.Div(children = [html.Img(id="dotplot",className="image", src = None, 
                                               style={'max-width': '100%','max-height':'60vh', 'display': 'inline-block'})],
                          id = 'html-dp_figure')

html_dp_button = html.Div([html.Button('Generate Plot', id='dp_button_press', n_clicks=0, className="btn btn-danger")],
                          style = {'display': 'none'},
                          id = 'html-dp_button')

####################################
####################################


html_dp_left_col = html.Div([html_drop_dp_obs,
                             html.Br(),
                             html_drop_dp_method,
                             html.Br(),
                             html_dp_minfoldchange,
                             html.Br(),
                             html_vmin_vmax,
                             html.Br(),
                             html_dp_var_type, #sub left
                             html_dp_obs_type, #sub right
                             html.Br(),html.Br(),
                             html_dp_n_genes,
                             html_dp_obs_order,
                             html.Br(),html.Br(),
                             html_dp_select_var,
                             html.Br(),html.Br(),
                             html_dp_select_obs,
                            ],
                            style={'width': '30%','height':'100%', 'float': 'left', 
                                   'display': 'inline-block','padding-left': '10px',
                                   'padding-bottom': '10px','padding-top': '10px','padding-right': '10px',
                                  },
                            className='bg-light',
                            id = 'html-dp_left_col')

html_dp_right_col = html.Div([html_dp_button,
                              html.Br(),
                              dcc.Loading([html_dp_figure],
                                          overlay_style={"visibility":"visible", "filter": "blur(2px)"},
                                          type="circle",),
                              #html_dp_figure,
                             ],
                             style={'width': '65%', 'float': 'right', 'display': 'inline-block','padding-left': '10px',
                                   'padding-bottom': '10px','padding-top': '10px','padding-right': '10px',},
                             id = 'html-dp_right_col')

tab2 = html.Div([html_dp_left_col,
                 html_dp_right_col, 
                ],
                style={'height':'100%'},
                id = 'html-tab2')

##############################################
##########  show items left tab    ###########
##############################################
@app.callback(
    Output('drop_dp_obs', 'options'),
    Output('html-drop_dp_obs','style'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_output_drop_dp_obs(anndata):
    return list(adata_dict[anndata].obs.dtypes[adata_dict[anndata].obs.dtypes =='category'].index), {'display': 'block'}
########################
@app.callback(
    Output('html-drop_dp_method','style'),
    Input('drop_dp_obs', 'value'),
    prevent_initial_call=True
)
def update_output_drop_dp_method(drop_dp_obs):
    if drop_dp_obs is not None:
        return {'display': 'block'}
########################
@app.callback(
    Output('html-dp_button','style'),
    Input('drop_dp_obs', 'value'),
    prevent_initial_call=True
)
def update_output_dp_button(drop_dp_obs):
    if drop_dp_obs is not None:
        return {'display': 'block'}
########################
@app.callback(
    Output('html-dp_minfoldchange','style'),
    Input('drop_dp_obs', 'value'),
    prevent_initial_call=True
)
def update_output_dp_minfoldchange(drop_dp_obs):
    if drop_dp_obs is not None:
        return {'display': 'block'}
########################
@app.callback(
    Output('html-dp_vmin_vmax','style'),
    Input('drop_dp_obs', 'value'),
    prevent_initial_call=True
)
def update_output_dp_vmin_vmax(drop_dp_obs):
    if drop_dp_obs is not None:
        return {'display': 'block'}
########################
@app.callback(
    Output('html-dp_var_type','style'),
    Input('drop_dp_obs', 'value'),
    prevent_initial_call=True
)
def update_output_dp_var_type(drop_dp_obs):
    if drop_dp_obs is not None:
        return {'width': '49%', 'float': 'left', 'display': 'inline-block','display': 'block'}
########################
@app.callback(
    Output('html-dp_obs_type','style'),
    Input('drop_dp_obs', 'value'),
    prevent_initial_call=True
)
def update_output_dp_obs_type(drop_dp_obs):
    if drop_dp_obs is not None:
        return {'width': '49%', 'float': 'right', 'display': 'inline-block','display': 'block'}
########################
@app.callback(
    Output('html-dp_n_genes','style'),
    Output('html-dp_select_obs','style'),
    Output('dp_select_obs','options'),
    Output('dp_select_obs','value'),
    Output('html-dp_select_var','style'),
    Output('dp_select_var','options'),
    Output('dp_minfoldchange','disabled'),
    Input('anndata', 'value'),
    Input('drop_dp_obs', 'value'),
    Input('dp_var_type', 'value'),
    prevent_initial_call=True
)
def update_output_dp_var_type_submenu(anndata,drop_dp_obs,dp_var_type):
    showstyle = {'width': '49%', 'float': 'left', 'display': 'inline-block','display': 'block'}
    hidestyle = {'width': '49%', 'float': 'left', 'display': 'inline-block','display': 'none'}
    if drop_dp_obs is not None:
        if dp_var_type == 'Automatic DEGs':
            new_options = list(adata_dict[anndata].obs[drop_dp_obs].cat.categories)
            new_value = list(adata_dict[anndata].obs[drop_dp_obs].cat.categories)
            return showstyle,showstyle,new_options,new_value,hidestyle,[],False
        elif dp_var_type == 'Specify Genes':
            new_options = adata_dict[anndata].var.index.tolist()
            return hidestyle,hidestyle,[],[],showstyle,new_options,True
        else:
            return hidestyle,hidestyle,[],[],hidestyle,[],False
    else:
        return hidestyle,hidestyle,[],[],hidestyle,[],False
########################
@app.callback(
    Output('html-dp_obs_order','style'),
    Output('dp_obs_order','rowData'),
    Output('dp_obs_order','columnDefs'),
    Input('anndata', 'value'),
    Input('drop_dp_obs', 'value'),
    Input('dp_obs_type','value'),
    prevent_initial_call=True
)
def update_output_dp_obs_order(anndata,drop_dp_obs,dp_obs_type):
    hidestyle = {'width': '49%', 'float': 'right', 'display': 'inline-block','display': 'none'}
    if drop_dp_obs is not None:
        if dp_obs_type == 'Customize order':
            showstyle = {'width': '49%', 'float': 'right', 'display': 'inline-block','display': 'block'}
            df_list = pd.DataFrame(list(adata_dict[anndata].obs[drop_dp_obs].cat.categories),columns=['order categories:'])
            rowData=df_list.to_dict("records")
            columnDefs=[{"field": i,'rowDrag': True} for i in df_list.columns]
            return showstyle,rowData,columnDefs
        else:
            return hidestyle,None,None
    else:
        return hidestyle,None,None

@app.callback(
    Output('dp_obs_order_store','data'),
    Input('dp_obs_type','value'),
    Input('dp_obs_order','virtualRowData'),
    prevent_initial_call=True
)
def update_obs_order_store(dp_obs_type,dp_obs_order):
    if dp_obs_type == 'Customize order':
        return dp_obs_order
    else:
        return None

##############################################
##############################################

##############################################
##########  generate right tab     ###########
##############################################

##############################################
##############################################

##########################################################
#################### app layout   ########################
##########################################################
app.layout = html.Div([
    ##############################################
    ####select anndata object from dictionary#####
    ##############################################
    html.B("Select object"),
    dcc.Dropdown(options = list(adata_dict), 
                 id="anndata",
                ),
    ##############################################

    ##############################################
    ###############tabular layout#################
    ##############################################
    dcc.Tabs([
        dcc.Tab(tab1, label="Interactive UMAP"),
        dcc.Tab(tab2, label="Dotplot",),
        ]),
    ##############################################
],style={'height':'100%'})
##########################################################
##########################################################


##############################################################################################################################################################################
##############################################################################################################################################################################
########################################################################################## Tab 1 callbacks ###################################################################
##############################################################################################################################################################################
##############################################################################################################################################################################

##########################################################
##########set anndata object for other menus##############
##########################################################

@app.callback(
    Output('dropdown_var', 'options'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_output(anndata):
    return adata_dict[anndata].var.index.tolist()

@app.callback(
    Output('dropdown_obs', 'options'),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_output(anndata):
    return adata_dict[anndata].obs.columns.tolist()

##########################################################
##########################################################

##########################################################
########## plot expression of gene on umap  ##############
##########################################################
@app.callback(
    Output("graph1", "figure"),
    Output('dropdown_var','value'),
    Input('anndata', 'value'),
    Input("slider_marker_size", "value"),    
    Input("dropdown_var", "value"),
    Input('deg_table', 'active_cell'),
    State('deg_table', 'derived_viewport_data'),
    prevent_initial_call=True
)

def update_graph_1(anndata,
                   slider_marker_size,
                   dropdown_var,
                   active_cell,
                   derived_viewport_data,
                  ):
    callback_id = ctx.triggered_id if not None else 'dropdown_var'
    if callback_id == 'dropdown_var':
        color_var = dropdown_var
    elif callback_id == 'deg_table':
        try:
            color_var = derived_viewport_data[active_cell['row']]['names']
        except:
            color_var = dropdown_var
    else:
        color_var = dropdown_var

    # if anndata not in adata_dict.keys():
    #     fig = None

    if anndata in adata_dict.keys():
        tmp_df = pd.DataFrame(adata_dict[anndata].obsm['X_umap'], columns = ['X','Y'])
        tmp_df.index = adata_dict[anndata].obs.index
        tmp_df = tmp_df.join(adata_dict[anndata].obs)
        tmp_df['barcode'] = list(tmp_df.index)
        plot_df = tmp_df
        fig = px.scatter(plot_df, x='X', y='Y',
                         color = adata_dict[anndata][:,color_var].X.toarray().flatten(),
                         hover_data={'X':False,
                                     'Y':False,
                                     'barcode':True,
                                     'sample':True,
                                     'phase':True,},
                         color_continuous_scale = plotly_reds,
                         labels = {'color':color_var},
                         width=700, height=700,
                        )
        fig.update_traces(marker={'size': slider_marker_size,
                         'line': {'width':.02,'color':'DarkSlateGrey'},
                                 })
        fig.update_layout(uirevision='constant')

    return fig, color_var

##########################################################
##########################################################

##########################################################
########## plot annotation of obs  on umap  ##############
##########################################################
@app.callback(
    Output("graph2", "figure"),
    Input("dropdown_obs", "value"),
    Input("slider_marker_size", "value"),
    Input('anndata', 'value'),
    prevent_initial_call=True
)
def update_graph_2(dropdown_obs, slider_marker_size,anndata):
    tmp_df = pd.DataFrame(adata_dict[anndata].obsm['X_umap'], columns = ['X','Y'])
    tmp_df.index = adata_dict[anndata].obs.index
    tmp_df = tmp_df.join(adata_dict[anndata].obs)
    tmp_df['barcode'] = list(tmp_df.index)
    plot_df = tmp_df
    sorted_list = []
    for i in plot_df:
        if pd.api.types.is_categorical_dtype(plot_df[i]):
            plot_df[i] = plot_df[i].cat.remove_unused_categories()
    if pd.api.types.is_categorical_dtype(plot_df[dropdown_obs]):
        try:
            sorted_list = sorted(list(plot_df[dropdown_obs].unique().categories), key = int)
        except:
            sorted_list = sorted(list(plot_df[dropdown_obs].unique().categories))
                             
     
    fig = px.scatter(plot_df, x='X', y='Y',
                        color = plot_df[dropdown_obs],
                        hover_data={'X':False,
                                    'Y':False,
                                    'barcode':True,
                                    'sample':True,
                                    'phase':True,},
                        #color='leiden_0.5',
                        color_continuous_scale = plotly_reds,
                        category_orders={dropdown_obs: sorted_list},
                        width=800, height=700,
                       )
    
    fig.update_traces(marker={'size': slider_marker_size,
                         'line': {'width':.02,'color':'DarkSlateGrey'},
                         })
    fig.update_layout(legend= {'itemsizing': 'constant'})

    fig.update_layout(uirevision='constant')
    return fig
##########################################################
##########################################################

##########################################################
#################### Update DEG table  ###################
##########################################################
@app.callback(Output('html-cluster_select','style'),
              Output('cluster_select','options'),
              Output('cluster_select','value'),
              Input('dropdown_obs','value'),
              Input('anndata', 'value'), 
              prevent_initial_call=True
             )
def cluster_dropdown(dropdown_obs,anndata):
    if 'wilcoxon_'+dropdown_obs in adata_dict[anndata].uns.keys():
        style = {'width': '49%', 'display': 'inline-block','display': 'block'}
        options = list(adata_dict[anndata].obs[dropdown_obs].cat.categories)
        value = list(adata_dict[anndata].obs[dropdown_obs].cat.categories)[0]
    else:
        style = {'width': '49%', 'display': 'inline-block','display': 'none'}
        options = []
        value = None
    return style,options,value

@app.callback(Output('html-deg_table','style'),
              Output('deg_table','data'),
              Output('deg_table','columns'),
              Input('dropdown_obs','value'),
              Input('anndata','value'),
              Input('cluster_select','value'),
              prevent_initial_call=True
             )
def update_datatable(dropdown_obs,anndata,cluster_select):            
    if 'wilcoxon_'+dropdown_obs in adata_dict[anndata].uns.keys():
        style = {'display': 'block'}
        deg_df = sc.get.rank_genes_groups_df(adata_dict[anndata], group=None, key='wilcoxon_'+dropdown_obs)
        deg_df = deg_df[deg_df['group']==cluster_select].iloc[:,1:6]
        data = deg_df.to_dict('records')
        columns =  [{"name": i, "id": i,} for i in (deg_df.columns)]
    else:
        style = {'display': 'none'}
        data = None
        columns = None
    return style,data,columns
##########################################################
##########################################################

##############################################################################################################################################################################
##############################################################################################################################################################################

##############################################################################################################################################################################
##############################################################################################################################################################################
########################################################################################## Tab 2 callbacks ###################################################################
##############################################################################################################################################################################
##############################################################################################################################################################################



##########################################################
############     generate dotplot         ################
##########################################################

@app.callback(
    Output('dotplot', 'src'),
   # Output('dp_button_press', 'n_clicks'),
    Input('dp_button_press', 'n_clicks'),
    State('anndata', 'value'),
    State('drop_dp_obs','value'),
    State('drop_dp_method','value'),
    State('dp_minfoldchange','value'),
    State('dp_vmin','value'),
    State('dp_vmax','value'),
    State('dp_var_type','value'),
    State('dp_n_genes','value'),
    State('dp_select_obs','value'),
    State('dp_select_var','value'), 
    State('dp_obs_type','value'),
    State('dp_obs_order_store','data'),
    State('dotplot', 'src'),
    prevent_initial_call=True
)
def generate_html_dp_figure(dp_button_press,
                            anndata,
                            drop_dp_obs,
                            drop_dp_method,
                            dp_minfoldchange,
                            dp_vmin,
                            dp_vmax,
                            dp_var_type,
                            dp_n_genes,
                            dp_select_obs,
                            dp_select_var,
                            dp_obs_type,
                            dp_obs_order_store,
                            dotplot_src,
                           ):
    if dp_button_press > 0:
        if drop_dp_method+'_'+drop_dp_obs not in adata_dict[anndata].uns:
            sc.tl.rank_genes_groups(adata_dict[anndata], 
                                    groupby = drop_dp_obs,
                                    method = drop_dp_method,
                                    key_added = drop_dp_method+'_'+drop_dp_obs,
                                   )
            
        if dp_obs_type == 'Automatic order':
            if 'dendrogram_'+drop_dp_obs not in adata_dict[anndata].uns:
                sc.tl.dendrogram(adata_dict[anndata],
                                 groupby = drop_dp_obs,
                                )
            categories_order = None
            dendrogram = True

        if dp_obs_type == 'Customize order':
            categories_order = [list(d.values())[0] for d in dp_obs_order_store]
            dendrogram = False

        if dp_var_type == 'Automatic DEGs':
            n_genes = dp_n_genes
            select_obs = dp_select_obs
            min_logfoldchange = dp_minfoldchange
            var_names = None

        if dp_var_type == 'Specify Genes':
            n_genes = None
            select_obs = None
            min_logfoldchange = None
            var_names = dp_select_var
            
        fig_buffer = io.BytesIO()
        dp = sc.pl.rank_genes_groups_dotplot(adata_dict[anndata], 
                                             groups = select_obs,
                                             groupby = drop_dp_obs,
                                             n_genes=n_genes,
                                             var_names = var_names,
                                             categories_order = categories_order,
                                             dendrogram = dendrogram,
                                             values_to_plot="logfoldchanges",
                                             cmap='bwr',
                                             vmin=dp_vmin,
                                             vmax=dp_vmax,
                                             min_logfoldchange=min_logfoldchange,
                                             key = drop_dp_method+'_'+drop_dp_obs,
                                             colorbar_title='log fold change',
                                             return_fig = True)
        dp.savefig(fig_buffer, format='svg')
        plt.close()
        encoded = base64.b64encode(fig_buffer.getvalue()).decode("utf-8")
        fig_src = "data:image/svg+xml;base64, " + encoded
        
        #n_clicks = 0
        #n_clicks = dp_button_press
    else:
        fig_src = dotplot_src
       # n_clicks = dp_button_press
    return fig_src
##########################################################
##########################################################

##############################################################################################################################################################################
##############################################################################################################################################################################

if __name__ == '__main__':
    app.run(jupyter_mode="tab",host='0.0.0.0', port = port ,debug=True)
print('Use this link: '+'http://cranio213:'+str(port))

In [ ]:
#annotation for E8
adata_dict['ME8']

cluster_anno = {
    '0':'Mesoderm - Cranial - Foxl2+',# seems to be anterior mesoderm https://www.cell.com/developmental-cell/fulltext/S1534-5807%2820%2930889-3
    '1':'Forebrain - Six3+',
    '2':'Midbrain - Pax5+',
    '3':'Neural Tube - Ccdc198+', #is 1700011H14Rik, also Cpn1 in same area in in situ https://bmcdevbiol.biomedcentral.com/articles/10.1186/1471-213X-7-92/tables/2
    '4':'Surface Ectoderm - Cranial - Epcam+',
    '5':'Neural Crest - Sox10+',
    '6':'Myocyte - Foxf1+', #could be myotome or lateral plate mesoderm. I don't know. Similar to heart but lacking heart markers. Has muscle markers.
    '7':'Somite - Meox1+',
    '8':'Hindbrain - Hoxa2+',
    '9':'Endoderm - Pax9+',
    '10':'Mesoderm - Pharyngeal - Tlx1+',
    '11':'Hindbrain - Vgll3+', #either rhombomere 1 or 2, difficult to say
    '12':'Endothelium - Tal1+',
    '13':'Floor Plate/Ventral Neural Tube - Nkx2-9+', #Nkx2-9, Shh
    '14':'Surface Ectoderm - Pharyngeal - Tbx1+',
    '15':'Myocyte - Heart - Myh6+',
    '16':'Strange cells - Pou5f1+', #Pou5f1 high but half of them have Hox8/9 expression. And these two groups stick together regardless of resolution.
    '17':'Mesoderm - Tbx6+',#strange, has tail markers, not sure what to think
    '18':'Notochord - T+',
    '19':'Surface Ectoderm - Wnt3',
}
adata.obs['annotation'] = adata.obs['leiden_post_QC'].map(cluster_anno)

adata.obs["annotation_ordered"] = pd.Categorical(
    values=adata.obs.annotation, 
    categories=['Mesoderm - Cranial - Foxl2+',
                'Mesoderm - Pharyngeal - Tlx1+',
                'Somite - Meox1+',
                'Mesoderm - Tbx6+',
                'Myocyte - Foxf1+',
                'Myocyte - Heart - Myh6+', 
                'Endothelium - Tal1+',
                'Forebrain - Six3+', 
                'Midbrain - Pax5+',
                'Floor Plate/Ventral Neural Tube - Nkx2-9+',
                'Hindbrain - Vgll3+',
                'Hindbrain - Hoxa2+',
                'Neural Tube - Ccdc198+',
                'Neural Crest - Sox10+',
                'Strange cells - Pou5f1+',
                'Surface Ectoderm - Cranial - Epcam+',
                'Surface Ectoderm - Pharyngeal - Tbx1+',
                'Surface Ectoderm - Wnt3',
                'Endoderm - Pax9+',
                'Notochord - T+',
               ], 
    ordered=True
)

adata.uns['annotation_ordered_colors'] = ["#bc7253",
"#d0da3f",
"#d54f2d",
"#929632",
"#d79636",
"#826f32",
"#dacd81"] + ["#36dee6",
"#9c31df",
"#45c6a1",
"#6145c5",
"#7482d8",
"#cf6bd8",
"#3d2a7a",
"#8a3f90"] +["#8c1543",
"#ff79eb",
"#eb0078",
"#f898df",
"#ba00a6"]


In [ ]:
#annotation for E9
cluster_anno = {
    '0':'Midbrain - Dorsal - Mab21l2+', #super clear expression https://www.sciencedirect.com/science/article/pii/S0925477399001276
    '1':'Midbrain-Hindbrain boundary - En2+',#En1/2 Fgf8/17, Sp8/9, Pax8
    '2':'Forebrain - Foxg1+',
    '3':'Mesoderm - Tbx1+',
    '4':'Hindbrain - Gbx1+',
    '5':'Mesenchyme - Frontonasal - Alx3+',
    '6':'Forebrain Dorsal - Rspo2+',
    '7':'Forebrain - Optic vesicle - Mitf+',
    '8':'Hindbrain - Casz1+',
    '9':'Mesoderm - Foxl2+',
    '10':'Surface Ectoderm - Trp63+',
    '11':'Mesenchyme - Maxillary - Dlx1+ Dlx5-',
    '12':'Mesenchyme - Mandibular - Dlx5+',
    '13':'Hindbrain - Ventral - Phox2b+',#Floor plate? part shh+, Foxa2+ part Hoxa2+ seems posterior floorplate
    '14':'Cranial Placodes - Six6+',
    '15':'Midbrain - Ventral - Foxa1+',#floor plate? shh+ and Otx2+, Dkk2 and spink1
    '16':'Forebrain - Ventral - Nkx2-4+',#Shh ventral diencephalon, Future hypothalamus? Paper in xenopus seems to indicate that Nkx2-1/4 overlap but 4 is more restricted to the ventral diencephalon
    '17':'Neural Crest - Sox10+',
    '18':'Endothelium - Emcn+',
    '19':'Neurons? - Myt1+', #myt1 neurod1 strong signal, half Otx2+ other half Hoxa2+, seems a mix of neurons from different origins
    '20':'Pharyngeal Pouch - Pax1+',
    '21':'Hindbrain - Vgll3+', #Mafb, Lmx1a, Tmem255a
    '22':'Mesoderm - Pharyngeal - Tlx1+',
    '23':'Trigeminal Ganglia - Tlx2+',
    '24':'Otic Placode - Sox10+',
    '25':'Mesoderm - Aldh1a2+',
    '26':'Myocyte - Tnni1+',
}
adata.obs['annotation'] = adata.obs['leiden_post_QC'].map(cluster_anno)

adata.obs["annotation_ordered"] = pd.Categorical(
    values=adata.obs.annotation, 
    categories=['Mesoderm - Foxl2+',
                'Mesoderm - Tbx1+',
                'Mesoderm - Pharyngeal - Tlx1+',
                'Mesoderm - Aldh1a2+',
                'Mesenchyme - Maxillary - Dlx1+ Dlx5-',
                'Mesenchyme - Mandibular - Dlx5+',
                'Mesenchyme - Frontonasal - Alx3+',
                'Neural Crest - Sox10+',
                'Myocyte - Tnni1+',
                'Surface Ectoderm - Trp63+',
                'Pharyngeal Pouch - Pax1+',
                'Cranial Placodes - Six6+',
                'Otic Placode - Sox10+',
                'Trigeminal Ganglia - Tlx2+',
                'Endothelium - Emcn+',
                'Hindbrain - Gbx1+',
                'Hindbrain - Casz1+',
                'Hindbrain - Vgll3+',
                'Hindbrain - Ventral - Phox2b+',
                'Neurons? - Myt1+',
                'Midbrain-Hindbrain boundary - En2+',
                'Midbrain - Dorsal - Mab21l2+',
                'Midbrain - Ventral - Foxa1+',
                'Forebrain Dorsal - Rspo2+',
                'Forebrain - Ventral - Nkx2-4+', 
                'Forebrain - Foxg1+',
                'Forebrain - Optic vesicle - Mitf+',
               ], 
    ordered=True
)
adata.uns['annotation_ordered_colors'] = ["#438560",
"#d7b034",
"#6fcda2",
"#62da41",
"#85722c",
"#65c263",
"#c3b96f",
"#578030",
"#a8c73e"]+["#be686d",
"#de8430",
"#d44468",
"#e09c72",
"#d84632",
"#98572f"]+["#4e519c",
"#63d6ad",
"#4c3dd4",
"#3f786b",
"#6970df",
"#85c3d1",
"#455e7e",
"#839dd8"]+["#a045cb",
"#ce99c8",
"#cf499e",
"#784371"]

sc.pl.umap(adata, color = ['annotation_ordered'])

In [ ]:
#annotation E10
#Things to note:
# - mesenchyme between the first (PA1, mandibular) and second pharyngeal arch(PA2) seems almost identical. The only difference is in the expression of Hoxa2.
#   https://www.sciencedirect.com/science/article/pii/S0012160615001876#f0005
#   The hoxa2+ mesenchyme is not separate from PA1 in our umap or clusters. The expression of all other genes at E10 seems the same.
#   Instead, the mesenchyme of PA1 and PA2 seems to separate together into clusters going from medial to lateral
# - The brain clusters can be subdivided into much smaller clusters


#At resolution 1.4
anno_leiden_1_4 = {
    '0':'Forebrain - Pallium - Emx2+', #Sp8, contains subclusters
    '1':'Frontonasal mesenchyme - Alx4+', #needs to be split
    '2':'Maxillary prominence - distal - Lhx8+',
    '3':'Mesoderm - Zic1+',
    '4':'Pharyngeal arches 1 & 2 - Proximal - Dlx5+',
    '5':'Pharyngeal arches 1 & 2 - Medial - Hand1+', #These arches don't separate, medial part of the arches.
    '6':'Forebrain - Optic vesicle - Mitf+', #also includes portion of ventral with Nkx2-4 + Shh, ideally split this with higher resolution
    '7':'Pharyngeal arch 1 - mandible - Pitx1+', #https://www.informatics.jax.org/assay/MGI:6188910#1A_id
    '8':'Hindbrain - Msx3+', #contains subclusters
    '9':'Mesoderm - Ebf2+',
    '10':'Frontonasal mesenchyme - Tfap2b-', #lacks tfap2b
    '11':'Hindbrain - Floor plate - Ntn1+', #contains subclusters
    '12':'Pharyngeal arches 1 & 2 - PA1 ventral & PA2 dorsal - Gsc+', #Where to ventral part of PA1 and dorsal part of PA2 touch
    '13':'Oral ectoderm - Pitx2+',
    '14':'Surface ectoderm - Wnt6+',
    '15':'Neurons - Forebrain - Foxg1+', #contains a bunch of different subclusters, ideally split at higher resolution
    '16':'Frontonasal mesenchyme - LNP - Pax7+',
    '17':'Neurons - Hindbrain - Hoxb2+',
    '18':'Maxillary prominence - proximal - Dlx1+',
    '19':'Blood - Hba-a2+',
    '20':'Olfactory placode - Sp8+',
    '21':'Forebrain - Ventral - subpallium - Olig2+',
    '22':'Frontonasal mesenchyme - Alx4+', #not a good cluster at this resolution
    '23':'Glia - Plp1+',
    '24':'Trigeminal - Tlx2+',
    '25':'Otic placode - Oc90+',
    '26':'Pharyngeal mesoderm - Tcf21+',
    '27':'Endothelial cells - Emcn+',
    '28':'Midbrain - thalamus? - Rspo3+', #also contains part of the forbrain subpallium mantle
    '29':'Mesoderm? - eye? - Pitx2+',
    '30':'Immune cells - Fcer1g+',
    '31':'Neurons - spinal cord? - Evx1+',
}

In [ ]:
#annotation E11
anno_leiden_1_4 = {
    '0':'Pharyngeal arches 1 & 2 - Hand2+',
    '1':'Pharyngeal arches 1 & 2 - Proximal - Dlx5+', #dlx5, dlx3/4
    '2':'Frontonasal - likely medial - Tfap2b+', #pou3f3
    '3':'Maxillary prominence? - dermal - Lef1+', #lhx8+ dlx5-
    '4':'Mandibular prominence - dermal - Lef1+', #dlx5
    '5':'Medial nasal mesenchyme - Lrriq1+',
    '6':'Palatal mesenchyme - Asb4+',
    '7':'Frontonasal mesenchyme - Cldn11+',
    '8':'Mandibular prominence - hinge - Osr2+',#dlx5 Osr1/2
    '9':'Frontonasal mesenchyme - LNP - Ebf1+',
    '10':'Frontonasal mesenchyme - LNP - Msx1+',
    '11':'Olfactory epithelium - Six3+',
    '12':'Dermal mesenchyme - Lef1+', #maybe fnp?
    '13':'Mesoderm - Ebf2+',
    '14':'Brain - neurons', #needs subclustering
    '15':'Forebrain - Foxg1+', #needs subclustering
    '16':'Surface ectoderm - Wnt6+',
    '17':'Maxillary prominence - Foxl2+',#weird cluster
    '18':'Mandibular prominence ? - Nrg1+',#no idea bad cluster
    '19':'Mesoderm - Zic1+',
    '20':'Hindbrain? - Msx3+',#needs subclustering
    '21':'Oral ectoderm - Pitx2+',
    '22':'Glia - Plp1+',
    '23':'Vomeronasal mesenchyme - Lgr5+',
    '24':'Endothelial cells - Emcn+',
    '25':'Trigeminal - Tlx2+',
    '26':'Otic placode - Oc90+',
    '27':'Olfactory neurons - Casr+', #casr, neurog1 etc.
    '28':'Muscle cells - Ttn+',
    '29':'Immune cells - Fcer1g+',
    '30':'Periderm - Rhov+', #i think
}

## notes
Annotating E12 is proving to be difficult. The mesenchyme is very hard to tell apart at the moment. I struggle to discern mandible for maxillary. The regional specification is getting lost.
The celltypes are becoming more clear, but the clustering is struggling a lot. Some clusters are still based on positional signals. Other clusters on celltype. Some mixed. it's very messy.

In [ ]:
#annotation E12
anno_leiden_1_4 = {
    '0':'Mesenchyme',
    '1':'Mesenchyme',
    '2':'Mesenchyme',
    '3':'Mesenchyme',
    '4':'Mesenchyme',
    '5':'Mesenchyme',
    '6':'Mesenchyme',
    '7':'Mesenchyme',
    '8':'Mesenchyme',
    '9':'Mesenchyme',
    '10':'Mesenchyme',
    '11':'Mesenchyme',
    '12':'Brain - Sox2+',#has subclusters
    '13':'Mesenchyme',
    '14':'Mesoderm - Zic1+',
    '15':'Oral ectoderm - Foxe1+',#definitely needs to be subdivided into more clusters
    '16':'Mesenchyme',
    '17':'Mesenchyme',
    '18':'Mesenchyme',
    '19':'Olfactory epithelium - Sp8+',#definitely needs to be subdivided into more clusters
    '20':'Mesenchyme',
    '21':'Mesenchyme',
    '22':'Brain - neurons - Nrn1+',#has subclusters
    '23':'Brain - neurons - Gad2+',#has subclusters
    '24':'Glia - Plp1+',
    '25':'Endothelial cells - Emcn+',
    '26':'Surface ectoderm - Wnt6+',
    '27':'Olfactory neurons - Atp8b4+',
    '28':'Muscle cells - Ttn+',
    '29':'Immune cells - Fcer1g+',
    '30':'Trigeminal - Tlx2+',
}

# anno_leiden_1 = {
#     '0':,
# }
# for cluster in anno_leiden_1:
#     adata_dict['ME12'].obs.loc[adata_dict['ME12'][adata_dict['ME12'].obs['leiden1'] == cluster].obs.index,'annotation'] = anno_leiden_1[cluster]